In [1]:
import pandas as pd
import numpy as np
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [2]:
url = "https://www.basketball-reference.com/leagues/NBA_2023.html#all_per_game_team-opponent"

In [3]:
html = urlopen(url)
soup = BeautifulSoup(html)

# DATAFRAME

In [4]:
for table in soup.find_all('table'):
    print(table.get('id'))

confs_standings_E
confs_standings_W
divs_standings_E
divs_standings_W
per_game-team
per_game-opponent
totals-team
totals-opponent
per_poss-team
per_poss-opponent
advanced-team
shooting-team
shooting-opponent


In [5]:
table = soup.find(lambda tag: tag.name =='table' and tag.get('id')=='shooting-team')

In [6]:
columns = table.findAll(lambda tag: tag.name=='tr',limit=2)
columns

[<tr class="over_header"> <th aria-label="" class="over_header center" colspan="7" data-stat=""></th> <th aria-label="" class="over_header center" colspan="6" data-stat="header_pct_fga">% of FGA by Distance</th><th></th> <th aria-label="" class="over_header center" colspan="6" data-stat="header_fg_pct">FG% by Distance</th><th></th> <th aria-label="" class="over_header center" colspan="2" data-stat="astd">% of FG Ast'd</th><th></th> <th aria-label="" class="over_header center" colspan="2" data-stat="dunks">Dunks</th><th></th> <th aria-label="" class="over_header center" colspan="2" data-stat="layups">Layups</th><th></th> <th aria-label="" class="over_header center" colspan="2" data-stat="header_corner">Corner</th><th></th> <th aria-label="" class="over_header center" colspan="2" data-stat="header_heaves">Heaves</th> </tr>,
 <tr> <th aria-label="Rk" class="ranker poptip sort_default_asc show_partial_when_sorting center" data-stat="ranker" data-tip="Rank" scope="col">Rk</th> <th aria-labe

In [7]:
headers = [th.getText() for th in columns[1].findAll('th')]
headers = headers[1:]
headers

['Team',
 'G',
 'MP',
 'FG%',
 'Dist.',
 '\xa0',
 '2P',
 '0-3',
 '3-10',
 '10-16',
 '16-3P',
 '3P',
 '\xa0',
 '2P',
 '0-3',
 '3-10',
 '10-16',
 '16-3P',
 '3P',
 '\xa0',
 '2P',
 '3P',
 '\xa0',
 '%FGA',
 'Md.',
 '\xa0',
 '%FGA',
 'Md.',
 '\xa0',
 '%3PA',
 '3P%',
 '\xa0',
 'Att.',
 'Md.']

In [8]:
rows = table.tbody.findAll('tr')[0:]

In [9]:
team_stats = [[td.getText() for td in rows[i].findAll('td')] for i in range(len(rows))]

In [10]:
df = pd.DataFrame(team_stats, columns = headers)
df.head()

,Team,G,MP,FG%,Dist.,,2P,0-3,3-10,10-16,...,Md.,,%FGA,Md.,,%3PA,3P%,,Att.,Md.
0,Atlanta Hawks,36,8715,.466,13.6,,.665,.252,.180,.128,...,186,,.249,456,,.237,.343,,3,0
1,Boston Celtics,36,8740,.479,15.6,,.521,.211,.195,.064,...,139,,.281,515,,.230,.381,,6,0
2,Brooklyn Nets,35,8425,.510,14.3,,.630,.232,.170,.135,...,174,,.223,386,,.231,.414,,2,0
3,Chicago Bulls,35,8500,.485,13.5,,.671,.282,.145,.113,...,149,,.283,495,,.267,.419,,2,0
4,Charlotte Hornets,36,8790,.447,12.9,,.646,.294,.211,.082,...,202,,.306,545,,.247,.364,,4,0


In [11]:
df.shape

(30, 34)

## Cleaning

In [12]:
df.isnull().sum()

Team     0
G        0
MP       0
FG%      0
Dist.    0
         0
2P       0
0-3      0
3-10     0
10-16    0
16-3P    0
3P       0
         0
2P       0
0-3      0
3-10     0
10-16    0
16-3P    0
3P       0
         0
2P       0
3P       0
         0
%FGA     0
Md.      0
         0
%FGA     0
Md.      0
         0
%3PA     0
3P%      0
         0
Att.     0
Md.      0
dtype: int64

In [13]:
df.dtypes

Team     object
G        object
MP       object
FG%      object
Dist.    object
         object
2P       object
0-3      object
3-10     object
10-16    object
16-3P    object
3P       object
         object
2P       object
0-3      object
3-10     object
10-16    object
16-3P    object
3P       object
         object
2P       object
3P       object
         object
%FGA     object
Md.      object
         object
%FGA     object
Md.      object
         object
%3PA     object
3P%      object
         object
Att.     object
Md.      object
dtype: object

In [14]:
df = df.drop('\xa0',axis=1)
df.columns

Index(['Team', 'G', 'MP', 'FG%', 'Dist.', '2P', '0-3', '3-10', '10-16',
       '16-3P', '3P', '2P', '0-3', '3-10', '10-16', '16-3P', '3P', '2P', '3P',
       '%FGA', 'Md.', '%FGA', 'Md.', '%3PA', '3P%', 'Att.', 'Md.'],
      dtype='object')

In [15]:
df.columns = ['Team', 'G', 'MP', 'FG%', 'Dist.', '% FGA 2P', '% FGA 0-3', '% FGA 3-10', '% FGA 10-16',
       '% FGA 16-3P', '% FGA 3P', 'FG% 2P', 'FG% 0-3', 'FG% 3-10', 'FG% 10-16', 'FG% 16-3P', 'FG% 3P', '% of FG Ast 2P', '% of FG Ast 3P',
       'Dunks %FGA', 'Dunks Md.', 'Layups %FGA', 'Layups Md.', 'Corner %3PA', 'Corner 3P%', 'Heaves Att.', 'Heaves Md.']

# FG% by Distance

In [46]:
df.sort_values(by=['FG% 2P'],ascending=False,inplace=True)
df.index = np.arange(1,len(df)+1)
df[['Team', 'FG% 2P', 'FG% 0-3', 'FG% 3-10', 'FG% 10-16', 'FG% 16-3P', 'FG% 3P']].head(3)

,Team,FG% 2P,FG% 0-3,FG% 3-10,FG% 10-16,FG% 16-3P,FG% 3P
1,Sacramento Kings,.588,.762,.500,.398,.425,.357
2,Brooklyn Nets,.580,.723,.464,.518,.526,.390
3,Minnesota Timberwolves,.578,.711,.459,.466,.427,.342


In [41]:
df.sort_values(by=['FG% 3P'],ascending=False,inplace=True)
df.index = np.arange(1,len(df)+1)
df[['Team', 'FG% 3P', 'FG% 0-3', 'FG% 3-10', 'FG% 10-16', 'FG% 16-3P', 'FG% 2P']].head(3)

,Team,FG% 3P,FG% 0-3,FG% 3-10,FG% 10-16,FG% 16-3P,FG% 2P
1,Denver Nuggets,.394,.745,.462,.395,.438,.569
2,Brooklyn Nets,.390,.723,.464,.518,.526,.580
3,Phoenix Suns,.386,.721,.413,.439,.408,.522


In [49]:
df.sort_values(by=['FG% 0-3'],ascending=False,inplace=True)
df.index = np.arange(1,len(df)+1)
df[['Team', 'FG% 0-3','FG% 2P', 'FG% 3-10', 'FG% 10-16', 'FG% 16-3P', 'FG% 3P']].head(3)

,Team,FG% 0-3,FG% 2P,FG% 3-10,FG% 10-16,FG% 16-3P,FG% 3P
1,Washington Wizards,.775,.559,.484,.425,.422,.347
2,Sacramento Kings,.762,.588,.500,.398,.425,.357
3,Denver Nuggets,.745,.569,.462,.395,.438,.394


In [53]:
df.sort_values(by=['FG% 3-10'],ascending=False,inplace=True)
df.index = np.arange(1,len(df)+1)
df[['Team', 'FG% 3-10','FG% 2P', 'FG% 0-3', 'FG% 10-16', 'FG% 16-3P', 'FG% 3P']].head(3)

,Team,FG% 3-10,FG% 2P,FG% 0-3,FG% 10-16,FG% 16-3P,FG% 3P
1,Golden State Warriors,.522,.563,.695,.458,.439,.376
2,Sacramento Kings,.500,.588,.762,.398,.425,.357
3,Dallas Mavericks,.499,.578,.703,.465,.402,.360


In [44]:
df.sort_values(by=['FG% 10-16'],ascending=False,inplace=True)
df.index = np.arange(1,len(df)+1)
df[['Team', 'FG% 10-16','FG% 2P', 'FG% 0-3', 'FG% 3-10', 'FG% 16-3P', 'FG% 3P']].head(3)

,Team,FG% 10-16,FG% 2P,FG% 0-3,FG% 3-10,FG% 16-3P,FG% 3P
1,Brooklyn Nets,.518,.580,.723,.464,.526,.390
2,Boston Celtics,.466,.575,.719,.475,.491,.376
3,Minnesota Timberwolves,.466,.578,.711,.459,.427,.342


# % of FGA Ast'd

- 2P

In [54]:
df.sort_values(by=['% of FG Ast 2P'],ascending=False,inplace=True)
df.index = np.arange(1,len(df)+1)
df[['Team', '% of FG Ast 2P', '% of FG Ast 3P']].head(3)

,Team,% of FG Ast 2P,% of FG Ast 3P
1,Golden State Warriors,.614,.816
2,Denver Nuggets,.597,.816
3,Indiana Pacers,.570,.806


- 3P

In [55]:
df.sort_values(by=['% of FG Ast 3P'],ascending=False,inplace=True)
df.index = np.arange(1,len(df)+1)
df[['Team', '% of FG Ast 3P', '% of FG Ast 2P']].head(3)

,Team,% of FG Ast 3P,% of FG Ast 2P
1,San Antonio Spurs,.944,.525
2,Chicago Bulls,.912,.473
3,Oklahoma City Thunder,.887,.441


# Corner

In [57]:
df.sort_values(by=['Corner %3PA'],ascending=False,inplace=True)
df.index = np.arange(1,len(df)+1)
df[['Team', 'Corner %3PA', 'Corner 3P%']].head(3)

,Team,Corner %3PA,Corner 3P%
1,Phoenix Suns,.289,.457
2,Dallas Mavericks,.270,.402
3,Chicago Bulls,.267,.419


In [56]:
df.sort_values(by=['Corner 3P%'],ascending=False,inplace=True)
df.index = np.arange(1,len(df)+1)
df[['Team', 'Corner 3P%', 'Corner %3PA']].head(3)

,Team,Corner 3P%,Corner %3PA
1,Phoenix Suns,.457,.289
2,Utah Jazz,.436,.206
3,Chicago Bulls,.419,.267
